In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [10]:
import numpy as np

def squarebounds(_open=None, close=None, alpha=0):

    N = len(close)

    up1, up2, dn1, dn2, bull, bear = np.zeros((6, N))

    up1[0] = dn1[0] = close[0]
    up2[0] = dn2[0] = close[0] ** 2
       
    for i in range(1, N):
        
        up1[i] = max(close[i], _open[i], up1[i - 1] - (up1[i - 1] - close[i]) / alpha)
        dn1[i] = min(close[i], _open[i], dn1[i - 1] + (close[i] - dn1[i - 1]) / alpha)

        up2[i] = max(close[i] ** 2, _open[i] ** 2, up2[i - 1] - (up2[i - 1] - close[i] ** 2) / alpha)
        dn2[i] = min(close[i] ** 2, _open[i] ** 2, dn2[i - 1] + (close[i] ** 2 - dn2[i - 1]) / alpha)
             
        
        bull[i] = np.sqrt(up2[i] - up1[i] ** 2)
        bear[i] = np.sqrt(dn2[i] - dn1[i] ** 2)
        
    bear *= -1
    
    bull_d1 = np.gradient(bull)
    bull_d2 = np.gradient(bull_d1)

    bear_d1 = np.gradient(bear)
    bear_d2 = np.gradient(bear_d1)
    
    long_condition = bull_d1[-1] < 0
    short_condition = bear_d1[-1] > 0
    
    return long_condition, short_condition

In [8]:
import priceanalytics.data as data

big_df = data.alpaca_download_single(
    'AAPL',     
    past_days=30, 
    interval=5  
)
dfs = data.split_alpaca_on_day(big_df)

In [15]:
import priceanalytics.backtest as bt
    
def test_strategy(df):
    
    with bt.Backtester(df) as b:
     
        for i, fr in b:
                      
            long, short = squarebounds(
                _open=fr['open'].to_numpy(),
                close=fr['close'].to_numpy(),
                alpha=10
            )
            
            if len(fr) <= 15: continue

            if long:
                b.buy()
            
            if short:
                b.sell()
            
        return b.results(1)

r = test_strategy(dfs[-7])
print(r)

-1.0
